In [ ]:
import os
import pickle
import copy
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

import matplotlib.pyplot as plt

## 0. Mount to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Customs-Declaration-Datasets-en/

/content/drive/MyDrive/Customs-Declaration-Datasets-en


In [ ]:
class label_encoder(object):
    def fit_pd(self,df,cols=[]):
        if len(cols) == 0:
            cols = df.columns
        self.class_index = {}
        for f in cols:
            uf = df[f].unique()
            self.class_index[f] = {}
            index = 1
            for item in uf:
                self.class_index[f][item] = index
                index += 1
    
    def fit_transform_pd(self,df,cols=[]):
        if len(cols) == 0:
            cols = df.columns
        newdf = copy.deepcopy(df)
        self.class_index = {}
        for f in cols:
            uf = df[f].unique()
            self.class_index[f] = {}
            index = 1
            for item in uf:
                self.class_index[f][item] = index
                index += 1
                
            newdf[f] = df[f].apply(lambda d: self.update_label(f,d))
        return newdf
    
    def transform_pd(self,df,cols=[]):
        newdf = copy.deepcopy(df)
        if len(cols) == 0:
            cols = df.columns
        for f in cols:
            if f in self.class_index:
                newdf[f] = df[f].apply(lambda d: self.update_label(f,d))
        return newdf
                
    def update_label(self,f,x):
        try:
            return self.class_index[f][x]
        except:
            self.class_index[f][x] = max(self.class_index[f].values())+1
            return self.class_index[f][x]

In [ ]:
def min_max_scaler(df, numerical_columns, scaler_path):
    df = df.loc[:, numerical_columns].copy()
    scalers = {}
    mm_sclaers = []
    # Set Minmax scaler
    if not os.path.exists(scaler_path):
        print('Make a New Min Max Scaler')

        # Scale for each Numeric Columns
        for col in numerical_columns:
            mm_sclaer = MinMaxScaler()

            # Fit the scaler
            mm_sclaer.fit(df.loc[:, [col]])
            mm_sclaers.append(mm_sclaer)

            # Normalize each value between 0~1 using the scaler
            df.loc[:, col] = mm_sclaer.transform(df.loc[:, [col]])
            scalers[col] = mm_sclaer

        # Store the encoder
        f = open(scaler_path, 'wb')
        pickle.dump(scalers, f)

    else:
        print('Normalize with existing Min Max Scaler')
        # load scalers
        f = open(scaler_path, 'rb')
        scalers = pickle.load(f)

        # Normalize each value between 0~1 using loaded scaler
        for col in numerical_columns:
            mm_scaler = scalers[col]
            df.loc[:, col] = mm_scaler.transform(df.loc[:, [col]])

    return df, scalers

## 1. Load Dataset  & concatenation

In [ ]:
df_train = pd.read_csv('./data/df_syn_train_eng.csv')

In [ ]:
df_valid = pd.read_csv('./data/df_syn_valid_eng.csv')
df_test = pd.read_csv('./data/df_syn_test_eng.csv')

In [ ]:
df_train

,Declaration ID,Date,Office ID,Process Type,Import Type,Import Use,Payment Type,Mode of Transport,Declarant ID,Importer ID,...,HS10 Code,Country of Departure,Country of Origin,Tax Rate,Tax Type,Country of Origin Indicator,Net Mass,Item Price,Fraud,Critical Fraud
0,97061800,2020-01-01,30,B,11,21,11,10,ZZR1LT6,QLRUBN9,...,4408909195,BE,BE,0.0,FEU1,G,108.0,372254.40,0,0
1,85945189,2020-01-01,40,B,11,21,11,40,SWF9S4X,7JD1S2X,...,6907221000,CN,CN,8.0,A,Y,11352.0,375751.20,0,0
2,77854751,2020-01-01,20,B,11,21,11,40,X4XT6P8,WI9O3I5,...,6208221000,CN,CN,5.2,FCN1,B,20.7,5353.02,0,0
3,46925060,2020-01-01,40,B,11,21,43,40,K7LCQTZ,6LI9721,...,9403501000,VN,VN,0.0,C,Y,9218.0,1477645.40,0,0
4,34131149,2020-01-01,20,B,11,21,11,10,1HMVIVH,RZ871V1,...,710807000,VN,VN,27.0,A,B,24000.0,6364800.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37380,78236084,2020-12-31,40,B,11,21,43,40,NZYZ967,GAF7CYG,...,3926909000,CN,CN,6.5,C,Y,573.0,343.80,1,1
37381,13057608,2020-12-31,20,B,93,31,43,10,RSUWWQ2,4H0M9JE,...,4016999000,CN,CN,4.8,FCN1,G,1938.0,35077.80,1,1
37382,32316450,2020-12-31,40,B,11,21,11,10,ZOAPZF5,GGYZ5JZ,...,8504405090,CN,CN,4.9,CIT,S,50.0,3105.00,0,0
37383,62223974,2020-12-31,20,B,11,21,11,40,09EO87L,JVSA120,...,7204210000,CA,CA,0.0,A,B,20502.0,311630.40,0,0


In [ ]:
df_train = df_train.astype({'Declaration ID':'str','Date':'datetime64', 'Office ID':'str', 'Process Type':'str', 
              'Import Type':'str', 'Import Use':'str', 'Payment Type':'str', 'Mode of Transport':'str', 
              'HS10 Code':'str', 'Fraud':'str', 'Critical Fraud':'str', 'Item Price':'float64'})
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37385 entries, 0 to 37384
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Declaration ID               37385 non-null  object        
 1   Date                         37385 non-null  datetime64[ns]
 2   Office ID                    37385 non-null  object        
 3   Process Type                 37385 non-null  object        
 4   Import Type                  37385 non-null  object        
 5   Import Use                   37385 non-null  object        
 6   Payment Type                 37385 non-null  object        
 7   Mode of Transport            37385 non-null  object        
 8   Declarant ID                 37385 non-null  object        
 9   Importer ID                  37385 non-null  object        
 10  Seller ID                    33670 non-null  object        
 11  Courier ID                   6714 non-nul

In [ ]:
df_valid = df_valid.astype({'Declaration ID':'str','Date':'datetime64', 'Office ID':'str', 'Process Type':'str', 
              'Import Type':'str', 'Import Use':'str', 'Payment Type':'str', 'Mode of Transport':'str', 
              'HS10 Code':'str', 'Fraud':'str', 'Critical Fraud':'str', 'Item Price':'float64'})
df_test=df_test.astype({'Declaration ID':'str','Date':'datetime64', 'Office ID':'str', 'Process Type':'str', 
              'Import Type':'str', 'Import Use':'str', 'Payment Type':'str', 'Mode of Transport':'str', 
              'HS10 Code':'str', 'Fraud':'str', 'Critical Fraud':'str', 'Item Price':'float64'})

## 2. Preprocess Data

In [ ]:
df_train = df_train.fillna(0)
df_valid = df_valid.fillna(0)
df_test = df_test.fillna(0)

## 4. Lable Encoding

In [ ]:
os.makedirs('./encoder/', exist_ok = True)

In [ ]:
# Numeric dataset preprocessing
numerical_columns = ['Net Mass', 'Item Price']
train_numerical_xs, scaler = min_max_scaler(df_train, numerical_columns, scaler_path='./encoder/min_max_scaler_eng.pkl')
valid_numerical_xs, _ = min_max_scaler(df_valid, numerical_columns, scaler_path='./encoder/min_max_scaler_eng.pkl')

Make a New Min Max Scaler
Normalize with existing Min Max Scaler


In [ ]:
X_train_discrete = df_train[['Office ID', 'Process Type', 'Import Type', 'Import Use',
               'Payment Type', 'Mode of Transport', 'Declarant ID', 'Importer ID',
               'Seller ID', 'Courier ID', 'HS10 Code', 'Country of Departure',
               'Country of Origin', 'Tax Rate', 'Tax Type',
               'Country of Origin Indicator']]

X_valid_discrete = df_valid[['Office ID', 'Process Type', 'Import Type', 'Import Use',
               'Payment Type', 'Mode of Transport', 'Declarant ID', 'Importer ID',
               'Seller ID', 'Courier ID', 'HS10 Code', 'Country of Departure',
               'Country of Origin', 'Tax Rate', 'Tax Type',
               'Country of Origin Indicator']]



y_train = df_train['Fraud']
y_valid = df_valid['Fraud']

In [ ]:
# Label encoder
encoder = label_encoder()
encoder.fit_pd(X_train_discrete)
encoder.transform_pd(X_train_discrete)

,Office ID,Process Type,Import Type,Import Use,Payment Type,Mode of Transport,Declarant ID,Importer ID,Seller ID,Courier ID,HS10 Code,Country of Departure,Country of Origin,Tax Rate,Tax Type,Country of Origin Indicator
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,2,1,1,1,1,2,2,2,2,1,2,2,2,2,2,2
2,3,1,1,1,1,2,3,3,3,1,3,2,2,3,3,3
3,2,1,1,1,2,2,4,4,4,1,4,3,3,1,4,2
4,3,1,1,1,1,1,5,5,5,1,5,3,3,4,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37380,2,1,1,1,2,2,129,5031,428,1,28,2,2,5,4,2
37381,3,1,4,6,2,1,329,7703,369,1,27,2,2,10,3,1
37382,2,1,1,1,1,1,337,158,1110,1,703,2,2,38,8,4
37383,3,1,1,1,1,2,110,12719,2370,1,506,11,9,1,2,3


In [ ]:
#save the encoder file
filename = './encoder/label_encoder.pkl'

output = open(filename,'wb')
pickle.dump(encoder, output)
output.close()

In [ ]:
# encode train, valid label encoder
encoding_train = encoder.transform_pd(X_train_discrete)
encoding_valid = encoder.transform_pd(X_valid_discrete)

In [ ]:
# concaternate ( label encoded data + MinMax scaled data)

X_train_concat = pd.concat([encoding_train, train_numerical_xs], axis=1)
X_valid_concat = pd.concat([encoding_valid, valid_numerical_xs], axis=1)

In [ ]:
X_train_concat

,Office ID,Process Type,Import Type,Import Use,Payment Type,Mode of Transport,Declarant ID,Importer ID,Seller ID,Courier ID,HS10 Code,Country of Departure,Country of Origin,Tax Rate,Tax Type,Country of Origin Indicator,Net Mass,Item Price
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1.799250e-06,5.283441e-05
1,2,1,1,1,1,2,2,2,2,1,2,2,2,2,2,2,1.891212e-04,5.333071e-05
2,3,1,1,1,1,2,3,3,3,1,3,2,2,3,3,3,3.448563e-07,7.597590e-07
3,2,1,1,1,2,2,4,4,4,1,4,3,3,1,4,2,1.535693e-04,2.097236e-04
4,3,1,1,1,1,1,5,5,5,1,5,3,3,4,2,3,3.998334e-04,9.033619e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37380,2,1,1,1,2,2,129,5031,428,1,28,2,2,5,4,2,9.546022e-06,4.879585e-08
37381,3,1,4,6,2,1,329,7703,369,1,27,2,2,10,3,1,3.228655e-05,4.978624e-06
37382,2,1,1,1,1,1,337,158,1110,1,703,2,2,38,8,4,8.329863e-07,4.406955e-07
37383,3,1,1,1,1,2,110,12719,2370,1,506,11,9,1,2,3,3.415577e-04,4.422999e-05


In [ ]:
os.makedirs('./data_enc/', exist_ok = True)
X_train_concat.to_csv('./data_enc/encoded_train_eng.csv', index=False)
X_valid_concat.to_csv('./data_enc/encoded_valid_eng.csv', index=False)

In [ ]:
y_train.to_csv('./data_enc/y_train_eng.csv', index=False)
y_valid.to_csv('./data_enc/y_valid_eng.csv', index=False)

In [ ]:
# preprocess test data

# Get fraud label
y_test = df_test['Fraud']

# scale numerical columns
numerical_columns = ['Net Mass', 'Item Price']
test_numerical_xs, _ = min_max_scaler(df_test, numerical_columns, scaler_path='./encoder/min_max_scaler_eng.pkl')

# encode categorical columns
X_test_discrete = df_test[['Office ID', 'Process Type', 'Import Type', 'Import Use',
               'Payment Type', 'Mode of Transport', 'Declarant ID', 'Importer ID',
               'Seller ID', 'Courier ID', 'HS10 Code', 'Country of Departure',
               'Country of Origin', 'Tax Rate', 'Tax Type',
               'Country of Origin Indicator']]

filename = './encoder/label_encoder.pkl'
pkl_file = open(filename, 'rb')
encoder = pickle.load(pkl_file) # reload the saved encoder file
pkl_file.close()
encoding_test = encoder.transform_pd(X_test_discrete)

X_test_concat = pd.concat([encoding_test, test_numerical_xs], axis=1)

Normalize with existing Min Max Scaler


In [ ]:
X_test_concat.to_csv('./data_enc/encoded_test_eng.csv', index=False)
y_test.to_csv('./data_enc/y_test_eng.csv', index=False)